# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233618917688                   -0.50    6.0
  2   -7.249693771322       -1.79       -1.39    1.0
  3   -7.250636235598       -3.03       -1.67    4.0
  4   -7.251009944710       -3.43       -1.88    2.0
  5   -7.251315585326       -3.51       -2.47    2.0
  6   -7.251338253070       -4.64       -3.22    1.0
  7   -7.251338780075       -6.28       -3.84    3.0
  8   -7.251338792206       -7.92       -4.09    2.0
  9   -7.251338798597       -8.19       -4.91    1.0
 10   -7.251338798696      -10.01       -5.54    3.0
 11   -7.251338798704      -11.11       -6.00    3.0
 12   -7.251338798704      -12.10       -6.74    2.0
 13   -7.251338798705      -13.38       -7.06    3.0
 14   -7.251338798705      -14.27       -7.73    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.027028853224363745
[ Info: Arnoldi iteration step 2: normres = 1.0076315356384133
[ Info: Arnoldi iteration step 3: normres = 0.43244372344646237
[ Info: Arnoldi iteration step 4: normres = 0.5081725780889584
[ Info: Arnoldi iteration step 5: normres = 0.20710649109718413
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.42e-02, 8.86e-02, 1.23e-01, 1.37e-01, 2.94e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5970060010860502
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.91e-03, 7.44e-02, 5.52e-01, 1.41e-01, 8.41e-02)
[ Info: Arnoldi iteration step 7: normres = 0.1614780198342718
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.66e-04, 1.13e-01, 6.13e-02, 6.45e-02, 6.46e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13066716774652196
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.91e